In [37]:
import pandas as pd
import numpy as np
dataset = pd.read_csv("list_attr_celeba.txt", sep = ' ', header=1, skiprows = 0)

data = np.vstack((dataset['Male'], dataset["Young"], dataset["Mustache"], (-dataset["No_Beard"]),\
                                                                dataset["Bald"]))

# np.where(data > 0, data, 0)
data=np.maximum(data, 0)
print("0:Male, 1:Young, 2: Mustache, 3:Beard, 4:Bald")
print(data.sum(axis=1))

0:Male, 1:Young, 2: Beard, 3:Mustache, 4:Bald
[ 84434 156734  33441   8417   4547]


In [56]:
data_ = data[:,np.where(data[3,:]==0)[0]]
print(data_.shape)
data__ = data_[:,np.where(data_[2,:]==1)[0]]
print(data__.shape)
# print(len(np.where(data__[4,:]==1)[0]))

(5, 194182)
(5, 25259)


In [5]:
import math
import os
import torch
import torch.distributions.constraints as constraints
import pyro
from pyro.optim import Adam
from pyro.infer import SVI, Trace_ELBO
import pyro.distributions as dist

n_steps = 100



# clear the param store in case we're in a REPL
pyro.clear_param_store()


def model(data):
    # sample f from the beta prior
    laten_sex = pyro.param('sex_latent', torch.tensor(0.5),
                   constraint=constraints.unit_interval)
    laten_mus_1 = pyro.param("latent_female_mustache", torch.tensor(0.5),
                   constraint=constraints.unit_interval)
    laten_mus_2 = pyro.param("latent_male_mustache", torch.tensor(0.5),
                   constraint=constraints.unit_interval)
    ff = [laten_mus_1,laten_mus_2]
    # loop over the observed data
    for i in pyro.plate("data_loop", data.shape[1]):
        # observe datapoint i using the bernoulli likelihood
        sex = pyro.sample('sex_{}'.format(i), dist.Bernoulli(laten_sex),obs = data[0,i])        
        pyro.sample("mustache_{}".format(i), dist.Bernoulli(ff[sex.long()]), obs=data[1,i])
        

def guide(data):
    pass

# setup the optimizer
adam_params = {"lr": 0.1, "betas": (0.80, 0.999)}
optimizer = Adam(adam_params)
pyro.clear_param_store()
# setup the inference algorithm
svi = SVI(model, guide, optimizer, loss=Trace_ELBO())

# do gradient steps
for step in range(n_steps):
    rand_list = np.random.randint(low = 0, high=data.shape[1], size=1000)
#     svi.step(data[:,rand_list.tolist()])
    svi.step(torch.tensor(data[:,rand_list.tolist()]).float())

    if(step %1 ==0):
        print('step: {}'.format(step))
        alpha0 = pyro.param("sex_latent")
        alpha1 = pyro.param("latent_female_mustache")
       
        alpha2 = pyro.param("latent_male_mustache")
        print(alpha0)
        print(alpha1)
        print(alpha2)




step: 0
tensor(0.4750, grad_fn=<ClampBackward>)
tensor(0.4750, grad_fn=<ClampBackward>)
tensor(0.4750, grad_fn=<ClampBackward>)
step: 1
tensor(0.4512, grad_fn=<ClampBackward>)
tensor(0.4503, grad_fn=<ClampBackward>)
tensor(0.4503, grad_fn=<ClampBackward>)
step: 2
tensor(0.4297, grad_fn=<ClampBackward>)
tensor(0.4259, grad_fn=<ClampBackward>)
tensor(0.4257, grad_fn=<ClampBackward>)
step: 3
tensor(0.4127, grad_fn=<ClampBackward>)
tensor(0.4022, grad_fn=<ClampBackward>)
tensor(0.4016, grad_fn=<ClampBackward>)
step: 4
tensor(0.4021, grad_fn=<ClampBackward>)
tensor(0.3792, grad_fn=<ClampBackward>)
tensor(0.3782, grad_fn=<ClampBackward>)
step: 5
tensor(0.3974, grad_fn=<ClampBackward>)
tensor(0.3571, grad_fn=<ClampBackward>)
tensor(0.3556, grad_fn=<ClampBackward>)
step: 6
tensor(0.3935, grad_fn=<ClampBackward>)
tensor(0.3358, grad_fn=<ClampBackward>)
tensor(0.3344, grad_fn=<ClampBackward>)
step: 7
tensor(0.3905, grad_fn=<ClampBackward>)
tensor(0.3154, grad_fn=<ClampBackward>)
tensor(0.3146, g

KeyboardInterrupt: 

In [3]:

inferred_model = infer_discrete(model, temperature=0, first_available_dim=-3)

for target_var in ["sex", "mustache"]:
    kwargs = {"sex_0": data["sex"].float()} if target_var == "mustache" else {"mustache_0": data["mustache"].float()}
    trace = poutine.trace(inferred_model).get_trace(**kwargs)
    if target_var == "sex":
        print('sex pred:', [float(f'{i:.2f}') for i in trace.nodes[target_var]["value"].flatten()])
    elif target_var == "b":
        print('female mustache pred:', f'{trace.nodes[target_var]["value"][0, :].mean():.2f}')
        print('male half mustache pred:', f'{trace.nodes[target_var]["value"][1, :].mean():.2f}')

NameError: name 'infer_discrete' is not defined